In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import edward as ed
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import networkx as nx

from edward.models import Normal, Poisson, InverseGamma
from observations import celegans
import collections

In [2]:
x_train = celegans("~/data")
#x_train = x_data[:148,:148]
#x_test = x_data[148:296,148:296]

In [3]:
from edward.models import Normal, Bernoulli, InverseGamma

N = x_train.shape[0]  # number of data points
K = 3  # latent dimensionality

scale_z = InverseGamma(alpha = tf.ones([1,K])*1e-3, beta = tf.ones([1,K])*1e-3) 
  
scale_b = InverseGamma(alpha = 1e-3, beta = 1e-3) #Gelman 2006

z2 = Normal(loc = tf.zeros([N, K]), scale = (tf.sqrt(scale_z)*tf.ones([N, K])))

z1 = Normal(loc = tf.zeros([N, K]), scale = (tf.sqrt(scale_z)*tf.ones([N, K])))
            
b = Normal(loc = tf.zeros(1), scale = (tf.sqrt(scale_b)*tf.ones(1)))


# Calculate N x N distance matrix.
# 1. Create a vector, [||z_1||^2, ||z_2||^2, ..., ||z_N||^2], and tile
# it to create N identical rows.
pi1 = tf.tile(tf.reduce_sum(tf.pow(z1, 2), 1, keepdims=True), [1, N])
pi2 = tf.tile(tf.reduce_sum(tf.pow(z2, 2), 1, keepdims=True), [1, N])        
           
# 2. Create a N x N matrix where entry (i, j) is ||z_i||^2 + ||z_j||^2
# - 2 z_i^T z_j.
pi = pi1 + tf.transpose(pi2) - 2 * tf.matmul(z1, z2, transpose_b=True)
# 3. minus pairwise distances and make rate along diagonals to
# be close to zero.
pi = -tf.sqrt(pi + tf.diag(tf.zeros(N) + 1e3))

pi = tf.sigmoid(pi + b) #med bias (mange eller få links)

x = Bernoulli(pi)

In [17]:
z1.eval()

array([[-6.16532893e+18, -9.80726659e+18, -1.06476397e+18],
       [ 1.33811445e+19,  1.29209516e+19, -3.47173050e+18],
       [-3.48964980e+18, -8.77954702e+18,  1.31383479e+17],
       [-1.34255634e+18, -3.54086202e+18, -1.21042060e+18],
       [ 8.66687897e+18, -2.34388446e+18, -5.05917736e+18],
       [ 2.76037782e+18,  1.21651011e+18,  6.16441854e+18],
       [ 9.83726566e+18, -1.31878242e+18, -2.08460808e+17],
       [ 1.95896936e+18,  1.22572182e+19, -1.13452200e+18],
       [-1.12885849e+18, -4.75453842e+18,  8.13316228e+18],
       [-1.63750825e+16, -2.98498715e+18, -4.63150636e+18],
       [ 9.80154803e+18,  9.35446571e+18,  7.51444235e+18],
       [ 1.49838674e+19,  2.63216147e+16, -8.87847118e+18],
       [ 6.92479776e+17, -1.76376838e+18,  1.42674347e+18],
       [-1.80109592e+19,  7.70340276e+18,  2.20019437e+18],
       [ 5.57365039e+18,  1.07337885e+18, -2.82266708e+18],
       [-9.79977671e+18,  7.70574857e+18, -2.85454879e+18],
       [ 1.35808938e+19,  4.65799745e+18

In [5]:
inference = ed.MAP([z1,z2,b], data={x: x_train})

/anaconda/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/anaconda/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


In [4]:
#tf.reset_default_graph()
qz1 = Normal(loc=tf.get_variable("qz1/loc", [N, K]),
             scale=tf.nn.softplus(tf.get_variable("qz1/scale", [N, K])))
qz2 = Normal(loc=tf.get_variable("qz2/loc", [N, K]),
             scale=tf.nn.softplus(tf.get_variable("qz2/scale", [N, K])))
qb = Normal(loc=tf.get_variable("qb/loc", 1),
             scale=tf.nn.softplus(tf.get_variable("qb/scale", 1)))

inference = ed.KLqp({z1: qz1, z2: qz2, b: qb}, data={x: x_train})

/anaconda/lib/python3.6/site-packages/edward/util/random_variables.py:52: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  not np.issubdtype(value.dtype, np.float) and \
/anaconda/lib/python3.6/site-packages/edward/util/random_variables.py:53: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  not np.issubdtype(value.dtype, np.int) and \


In [5]:
inference.run(n_iter=2500)

2500/2500 [100%] ██████████████████████████████ Elapsed: 19s | Loss: nan